# GroupBy机制
- 'split-apply-combine'（拆分-应用-合并）
- [待分组数据].groupby([分组键])：
    - 返回的是一个GroupBy对象，含有一些有关分组键的中间数据，并未进行任何实际计算，再调用方法才能实现计算，计算结果返回一个Series；
    - 若传入多个分组键的列表，则计算结果返回一个层次化索引的Series；
    - 分组键可以是Series，或是数组，或是列名，或是df.dtypes，或是字典，或是索引级别level=；
    - 若不指定具体的列为待分组数据，则默认对所有数值列聚合；
    - 默认在axis=0上分组，可设置axis=1对列分组；
    - 可以存在未使用的分组键；
    - 会默认滤除缺失值；
    - groupby中传入as_index=False，可取消返回结果中的层次化索引；
    - groupby中传入group_keys=False，可不显示分组键一列；
- GroupBy的size方法，返回一个含有分组大小的Series；
- 支持对分组进行迭代，产生一组由分组名和数据块组成的二元元组
    - for name, group in df.groupby('xxx'):
    - for (name1, name2), group in df.groupby('xxx'):
    - 常将数据片段做成字典再进行其他操作：pieces = dict(list(df.groupby('xx')))
- 选取一列或列的子集
    - 直接对GroupBy对象进行列名的索引，即可选取部分列；
    - df.groupby('key')['col']等价于df['col'].groupby(df['key'])

# 数据聚合函数grouped.agg(func)
- 聚合指的是任何能够从数组产生标量值的数据转换过程，如mean、count等；
- 若使用自定义聚合函数func，则grouped.agg(func)；
- 可传递一组聚合函数，分别对所有数据分组操作，grouped.agg([func, 'mean', 'std'])；
- 也可对不同列传递不同的聚合操作，则需传入由(name, func)元组组成的列表，grouped.agg([('col1', 'func1'), ('col2', 'func2')])；

# 方法grouped.apply(func)
- 将待处理的对象拆分成多个片段，然后对各片段调用传入的函数，最后尝试将各片段组合到一起；（除聚合函数外的任何操作？）
- data.groupby('key').apply(func)

# 透视表和交叉表
- 透视表frame.pivot_table(['col1', 'col2'], index=, columns=)
    - ['col1', 'col2']为frame中待处理的特定对象，可省略（则针对所有数据）；
    - index和columns设置透视表中的行和列
    - 传入margins=True，添加分项小计，结果中将添加标签为All的行和列，其值对应于单个等级中所有数据的分组统计，默认是平均值（即aggfunc='mean'）
    - 传入aggfunc=，使用其他的聚合函数，结果中All对应相应的意义
    - 传入fill_value=，替换NA
- 交叉表pd.crosstab(data1, data2, margins=)
    - 一种用于计算分组频率的特殊透视表
    - data1和data2分别表示表中的行和列
    - 等效于data.pivot_table(..., aggfunc='count')

In [9]:
import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

#按key1分组，并计算data1列的平均值
grouped = df['data1'].groupby(df['key1'])
grouped
grouped.mean()

means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
means.unstack()

states = np.array(['ohio', 'LA', 'LA', 'ohio', 'ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [23]:
#对分组迭代

import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

for name, group in df.groupby('key1'):
    print(name)
    print(group)
print('\n***********************\n')
    
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
print('\n***********************\n')
    

pieces = dict(list(df.groupby('key1')))
pieces
pieces['b']

df.dtypes
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.882924  0.135813
1    a  two  2.085857 -1.478586
4    a  one -0.068375 -0.802419
b
  key1 key2     data1     data2
2    b  one -1.307165 -1.242915
3    b  two -3.138901  1.713752

***********************

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.882924  0.135813
4    a  one -0.068375 -0.802419
('a', 'two')
  key1 key2     data1     data2
1    a  two  2.085857 -1.478586
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.307165 -1.242915
('b', 'two')
  key1 key2     data1     data2
3    b  two -3.138901  1.713752

***********************

float64
      data1     data2
0  0.882924  0.135813
1  2.085857 -1.478586
2 -1.307165 -1.242915
3 -3.138901  1.713752
4 -0.068375 -0.802419
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [36]:
#选取某列

import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

df.groupby('key1')['data1'].mean()
df['data1'].groupby(df['key1']).mean()

df.groupby(['key1', 'key2'])[['data2']].mean()
df[['data2']].groupby(df['key1']).mean()

data2
key1 key2          
a    one  -0.718688
     two   1.058582
b    one   0.564204
     two  -1.195398

In [46]:
# 分组键

import pandas as pd
import numpy as np
from pandas import DataFrame, Series

people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
#print(people)

# 通过字典分组
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

by_column = people.groupby(mapping, axis=1)
by_column.sum()

# 通过Series分组
map_series = pd.Series(mapping)
people.groupby(map_series, axis=1).count()

# 通过函数分组
people.groupby(len).sum()  #字符串长度

# 混合
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

# 根据索引级别
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [61]:
# 聚合

import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

#使用自定义的聚合函数
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

#取消层次化索引
df.groupby(['key1', 'key2'], as_index=False)['data1'].mean()


data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.098763  0.323859
     std    0.797024  1.663600
     min   -1.007679 -1.590330
     25%   -0.388543 -0.224469
     50%    0.230592  1.141392
     75%    0.355696  1.280953
     max    0.480799  1.420515
b    count  2.000000  2.000000
     mean  -0.499461 -0.725712
     std    1.185658  0.583856
     min   -1.337848 -1.138561
     25%   -0.918655 -0.932136
     50%   -0.499461 -0.725712
     75%   -0.080268 -0.519287
     max    0.338926 -0.312863

In [64]:
# apply

import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
grouped = df.groupby('key1')

grouped.describe()

f = lambda x: x.describe()
grouped.apply(f)

#禁止分组键
df.groupby('key1', group_keys=False).apply(f)


data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -1.351185 -1.152856
     std    0.864666  0.355792
     min   -2.307388 -1.552732
     25%   -1.714639 -1.293637
     50%   -1.121891 -1.034543
     75%   -0.873084 -0.952919
     max   -0.624277 -0.871295
b    count  2.000000  2.000000
     mean  -0.779254  0.160950
     std    0.610114  1.137562
     min   -1.210669 -0.643428
     25%   -0.994961 -0.241239
     50%   -0.779254  0.160950
     75%   -0.563546  0.563139
     max   -0.347838  0.965328

In [72]:
# 透视表

import numpy as np
import pandas as pd
from pandas import DataFrame

data = pd.DataFrame({'sample': range(6),
                     'nationality': ['USA', 'Japan', 'USA', 'Japan', 'USA', 'USA'],
                     'handedness': ['r', 'l', 'r', 'r', 'l', 'r'],
                     'val': [2, 3, 6, 1, 3, 3]})
pd.crosstab(data.nationality, data.handedness, margins=True)

data.pivot_table('val', index='nationality', columns='handedness', aggfunc='count', margins=True)

handedness,l,r,All
nationality,,,
Japan,1,1,2
USA,1,3,4
All,2,4,6
